In [ ]:
import json
import requests
import numpy as np
import pytz
import pandas as pd
import chardet

url = 'https://raw.githubusercontent.com/SEBASBELMOS/KPI-Tech/main/salida.json'
response = requests.get(url)
encoding = chardet.detect(response.content)['encoding']
data = response.content.decode(encoding)

tickets = [json.loads(line) for line in data.splitlines()]

df = pd.json_normalize(tickets)

In [ ]:
columnas_a_eliminar = ['_id', 'departamento', 'fecha_creacion', 'prioridad', 'cliente._id', 'cliente.numero_identificacion', 'contacto.cargo', 'contacto.nombre', 'creador.cargo.__v', 'creador.cargo.consecutivo', 'creador.cargo.valor', 'creador.cargo.categoria', 'creador.cargo._id', 'creador.nombre', 'responsable._id', 'responsable.proceso._id', 'responsable.proceso.valor', 'responsable.tipo', 'responsable', 'responsable.proceso', 'creador.cargo', 'solucion.medio', 'solucion.medio._id', 'contacto', "tipo_soporte", "responsable.numero_identificacion","responsable.cargo"]
df = df.drop(columnas_a_eliminar, axis=1)

In [ ]:
df['responsable.nombre'] = df['responsable.nombre'].str.title()


df['responsable.nombre'].replace({
    'Santiago Ruiz Velasco': 'SANTIAGO RUIZ VELASCO',
    'Leydi Calad Blanco': 'LEIDY CALAD BLANCO'
}, inplace=True)

print(df['responsable.nombre'].unique())

['Jennifer Valencia Esquivel' 'Lina Marcela Vargas Quintero'
 'Bernardo Ruiz Caldas' 'Paul Andre Ruiz Caldas'
 'Silvana Irene Bonilla Llano' nan 'Leidy Calad Blanco'
 'SANTIAGO RUIZ VELASCO' 'Andres Felipe Vera Cifuentes'
 'Comptel System Undefined' 'Undefined Undefined' 'LEIDY CALAD BLANCO'
 'Luisa Maria Atehortua' 'Jhonatan Galvis'
 'Juliana Katherine Pedraza Cardenas' 'Tivisay Rubiano Quintero'
 'Miguel 0' 'Nathalia Rivera Posso' 'Jean Carlos Quisoboni Jimenez'
 'Maria Victoria Murillo Prado']


In [ ]:
frecuencia_valores = df['responsable.nombre'].value_counts()
print(frecuencia_valores)

In [ ]:
df = df.drop(df.index[46584:])

In [ ]:
df = df.dropna()

In [ ]:
print(df.isna().any())

detalle                  False
estado                   False
titulo                   False
cliente.nombre           False
fecha_server.$date       False
responsable.nombre       False
solucion.fecha_inicio    False
solucion.fecha_fin       False
solucion.medio.valor     False
solucion.detalle         False
dtype: bool


In [ ]:
f_df = df.copy()

f_df['fecha_server.$date'] = pd.to_datetime(f_df['fecha_server.$date'])


colombia_tz = pytz.timezone('America/Bogota')
f_df['fecha_server.$date'] = f_df['fecha_server.$date'].dt.tz_convert(colombia_tz)


f_df['fecha_server.$date'] = pd.to_datetime(f_df['fecha_server.$date'], format='%Y-%m-%d %H:%M:%S')


colombia_tz = pytz.timezone('America/Bogota')


f_df['fecha_server.$date'] = f_df['fecha_server.$date'].dt.tz_convert(colombia_tz).dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
f_df.to_csv('fuente.csv')

In [ ]:
new_df = df[['fecha_server.$date', 'solucion.fecha_inicio', 'solucion.fecha_fin','responsable.nombre']].copy()

new_df['fecha_server.$date'] = pd.to_datetime(new_df['fecha_server.$date'])


colombia_tz = pytz.timezone('America/Bogota')

new_df['fecha_server.$date'] = new_df['fecha_server.$date'].dt.tz_convert(colombia_tz)


new_df['fecha_server.$date'] = pd.to_datetime(new_df['fecha_server.$date'], format='%Y-%m-%d %H:%M:%S')


colombia_tz = pytz.timezone('America/Bogota')


new_df['fecha_server.$date'] = new_df['fecha_server.$date'].dt.tz_convert(colombia_tz).dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
d_df = new_df.copy()

d_df = d_df.astype(str)

In [ ]:
d_df['fecha_server.$date'] = pd.to_datetime(d_df['fecha_server.$date'], format="%Y-%m-%d %H:%M:%S", errors='coerce')
d_df['solucion.fecha_inicio'] = pd.to_datetime(d_df['solucion.fecha_inicio'], format="%Y-%m-%d %H:%M:%S", errors='coerce')
d_df['solucion.fecha_fin'] = pd.to_datetime(d_df['solucion.fecha_fin'], format="%Y-%m-%d %H:%M:%S", errors='coerce')

d_df = d_df.dropna()

In [ ]:
d_df.to_csv('fechas_responsables.csv')